# Google Colab Rust Setup

The following cell is used to set up a Rust environment on Colab. Don't execute it locally!

In [ ]:
# This script sets up and spins up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

# Gems Usage

This Jupyter Notebook provides a guide on using the [**`gems`**](https://github.com/wiseaidev/gems) cli and SDK in Rust to interact with the Gemini API. The [**Gemini API**](https://ai.google.dev/docs) allows you to generate creative content using large language models. In this notebook, we'll cover various aspects of using Gemini, including text-only input, text-and-image input, multi-turn conversations, configuration options, streaming content generation, counting tokens, embedding, and model information.

Let's start by installing the sdk to run the code snippets in this notebook.

In [3]:
:dep gems = { version = "0.1.0" }

---

## Import Client

In [4]:
use gems::Client;
use gems::traits::CTrait;

## API Key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in [Google AI Studio](https://makersuite.google.com/app/apikey).

In [5]:
fn config() -> (String, String) {
    let api_key = "your-api-key";
    let model = "gemini-2.0-flash";
    (api_key.to_string(), model.to_string())
}

## 1. Text-Only Input

To generate content using only text input, you can use the **`chat`** method with the `gemini-2.0-flash` model. Replace `Your_API_Key` above with your actual API key.

In [19]:
use gems::messages::Content;
use gems::messages::Message;
use gems::models::Model;
use gems::chat::ChatBuilder;


let mut gemini_client = Client::builder().model(&config().1).build()?;

gemini_client.set_api_key(config().0);

let parameters = ChatBuilder::default()
    .model(Model::Flash20)
    .messages(vec![Message::User {
        content: Content::Text("Hello".to_string()),
        name: None,
    }])
    .build()?;

match gemini_client.chat().generate(parameters).await {
    Ok(response) => {
        println!("{}", response);
    }
    Err(err) => {
        eprintln!("Error: {:?}", err);
    }
}

Error: Subprocess terminated with status: signal: 9 (SIGKILL)

## 2. Streaming Content

By default, the model returns a response after completing the entire generation process. You can achieve faster interactions by not waiting for the entire result and instead use streaming to handle partial results.

In [ ]:
use gems::stream::StreamBuilder
use gems::messages::Content;
use gems::messages::Message;

let input_text = "Write a story about a magic backpack.";
            
let parameters = StreamBuilder::default()
    .model(Model::Flash20)
    .input(Message::User {
        content: Content::Text(input_text.to_string()),
        name: None,
    })
    .build()?;

match gemini_client.stream().generate(parameters).await {
    Ok(response) => {
        println!("{:?}", response);
    }
    Err(err) => {
        eprintln!("Error: {:?}", err);
    }
}

## 3. Counting Tokens

When using long prompts, it might be useful to count tokens before sending any content to the model.

In [ ]:
use gems::tokens::TokenBuilder
use gems::messages::Content;
use gems::messages::Message;

let input_text = "Write a story about a magic backpack.";
        
let params = TokenBuilder::default()
    .input(Message::User {
        content: Content::Text(input_text.to_string()),
        name: None,
    })
    .build()?;

let count = gemini_client.tokens().count(params).await?;

count

## 4. Embedding

Embedding is a technique used to represent information as a list of floating point numbers in an array. With `gems`, you can represent text in a vectorized form. The following code demonstrates embedding text.

In [ ]:
use gems::embed::EmbeddingBuilder;
use gems::models::Model;

let input_text = "Hello";
        
let params = EmbeddingBuilder::default()
    .model(Model::Embedding)
    .input(Message::User {
        content: Content::Text(input_text.to_string()),
        name: None,
    })
    .build()?;

gemini_client.set_model(Model::Embedding);
let response = gemini_client.embeddings().create(params).await?;

response

## 5. Model Information

Your can retrieve information about the current model being used by the client.

In [ ]:
use gems::models::Model;
use gems::models::ModBuilder;

let params = ModBuilder::default().model(Model::default()).build()?;
let model_info = gemini_client.models().get(params).await?;

model_info.print();

## 6. Models Information

You can retrieve information about all available models.

In [ ]:
let models = gemini_client.models().list().await?;

models.print();

---

## Gems CLI
### Shell Command Execution

In [2]:
use std::process::{Command, Output, Stdio};

// A helper function to execute a shell command from a Rust script
fn execute_command(command: &str) -> Result<(), std::io::Error> {
    let status = Command::new("bash")
        .arg("-c")
        .arg(command)
        .stderr(Stdio::inherit())
        .status()?;

    if status.success() {
        Ok(())
    } else {
        Err(std::io::Error::from_raw_os_error(status.code().unwrap_or(1)))
    }
}

In [3]:
let command = "../target/release/gems --help";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

gems 0.0.9

💎 GEMS

A command-line tool for interacting with the Google Gemini API.

FUNCTIONALITIES:
  - Generate Content: Generate content using the Gemini API.
  - Stream Generate Content: Stream content generation using the Gemini API.
  - Count Tokens: Count tokens in a given text using the Gemini API.
  - Embed Content: Embed content using the Gemini API.
  - Batch Embed Contents: Batch embed contents using the Gemini API.
  - Get Model Info: Get information about the model from the Gemini API.
  - List Models: List available models from the Gemini API.

USAGE:
  gems [OPTIONS] <COMMAND>

EXAMPLES:
  Generate content:
    gems generate -t "Hello"

  Stream generate content:
    gems stream -t "Generate a short fictional story"

  Vision:
    gems vision -i /path/to/your/image.jpg

  Count tokens:
    gems count -t "Hello There!"

  Embed content:
    gems -m 'embedding-001' embed -t "Write a story about a magic backpack."

  Batch embed contents:
    gems -m 'embedding-001' batch

()

  info      
  list      
  help      Print this message or the help of the given subcommand(s)

Options:
  -a, --api-key <API_KEY>  API key for authentication
  -m, --model <MODEL>      Model to be used
  -h, --help               Print help
  -V, --version            Print version

AUTHORS:
    Mahmoud Harmouch


## 1. Text-Only Input

In [4]:
let command = "GEMINI_API_KEY=Your_API_Key ../target/release/gems generate -t hello";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

// Replace Your_API_Key above with your actual API key

Hello! How can I help you today?



()

## 2. Streaming Content

In [5]:
let command = "GEMINI_API_KEY=Your_API_Key ../target/release/gems stream -t 'Generate a short fictional story'";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

// Replace Your_API_Key above with your actual API key

The chipped porcelain teacup trembled in Elara’s hand, mirroring the anxiety churning in her stomach. Outside, the perpetual rain of Aethelgard hammered against the stained-glass windows of the sanctuary, a melancholic percussion accompanying her confession.

“It started subtly, Reverend,” she whispered, her voice barely audible above the storm. “With the shadows. Just a flicker in the corner of my eye, a fleeting sense of…being watched.”

Reverend Silas, a man whose wrinkles held the weight of countless confessions, simply nodded, his gaze fixed on the intricate carvings of the altar.

“Then, the dreams. Vivid, unsettling. I walk through the whispering woods, always searching for something…or someone. And the trees…their branches writhe like grasping fingers.” Elara shivered, pulling her shawl tighter.

For generations, the women of Aethelgard had been blessed – or cursed – with the Sight. Visions, prophecies, snippets of the future. But Elara's gift felt different, darker.

s express

()

## 3. Counting Tokens

In [6]:
let command = "GEMINI_API_KEY=Your_API_Key ../target/release/gems count -t 'Hello There!'";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

// Replace Your_API_Key above with your actual API key

Token Count: 3


()

## 4. Embedding

In [7]:
let command = "GEMINI_API_KEY=Your_API_Key ../target/release/gems -m 'embedding-001' embed -t 'hello'";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

// Replace Your_API_Key above with your actual API key

Embed Content: EmbedContentResponse { embedding: Some(Embedding { values: [0.048244964, 0.011776606, -0.0115520675, -0.018164538, -0.002611024, 0.050926752, 0.08172899, 0.007869764, 0.054475933, 0.026131338, -0.06593486, -0.0022568672, 0.03878191, 0.010530227, -0.04230794, -0.038269036, -0.0098457895, -0.00757351, -0.08022082, -0.03147243, 0.0022871997, -0.0012856963, 0.006200954, -0.03950943, -0.01869138, 0.045126893, 0.004105777, -0.0033850763, 0.023997491, -0.012320019, -0.03423185, 0.04703393, -0.022898493, -0.047150094, 0.00780749, 0.01974024, -0.043441188, 0.025867185, 0.0030507015, -0.08875779, -0.025923807, 0.06206684, 0.014752337, -0.0058687106, 0.020233197, 0.0046556373, 0.09399772, 0.045711495, -0.003133404, 0.017434591, 0.05630074, 0.011448362, -0.07785574, 0.010087488, 0.02003082, 0.015717002, -0.007205217, -0.030741267, 0.12710878, 0.0029991518, -0.032721598, -0.033147562, 0.02915778, -0.0074991942, 0.003163215, -0.015940316, 0.004189512, 0.015689038, -0.028875796, -0.019

()

## 5. Model Information

In [8]:
let command = "GEMINI_API_KEY=Your_API_Key ../target/release/gems info";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

// Replace Your_API_Key above with your actual API key


Model:
    Name: gemini-2.0-flash
    Version: 2.0
    Display Name: Gemini 2.0 Flash
    Description: Gemini 2.0 Flash
    Input Token Limit: 1048576
    Output Token Limit: 8192
    Supported Generation Methods: ["generateContent", "countTokens"]
    Temperature: Some(1.0)
    Max Temperature: Some(2.0)
    Top P: Some(0.95)
    Top K: Some(40)



()

## 6. Models Information

In [9]:
let command = "GEMINI_API_KEY=Your_API_Key ../target/release/gems list";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

// Replace Your_API_Key above with your actual API key.


Model:
    Name: chat-bison-001
    Version: 001
    Display Name: PaLM 2 Chat (Legacy)
    Description: A legacy text-only model optimized for chat conversations
    Input Token Limit: 4096
    Output Token Limit: 1024
    Supported Generation Methods: ["generateMessage", "countMessageTokens"]
    Temperature: Some(0.25)
    Max Temperature: None
    Top P: Some(0.95)
    Top K: Some(40)


Model:
    Name: text-bison-001
    Version: 001
    Display Name: PaLM 2 (Legacy)
    Description: A legacy model that understands text and generates text as an output
    Input Token Limit: 8196
    Output Token Limit: 1024
    Supported Generation Methods: ["generateText", "countTextTokens", "createTunedTextModel"]
    Temperature: Some(0.7)
    Max Temperature: None
    Top P: Some(0.95)
    Top K: Some(40)


Model:
    Name: embedding-gecko-001
    Version: 001
    Display Name: Embedding Gecko
    Description: Obtain a distributed representation of a text.
    Input Token Limit: 1024
    Outp

()

    Top K: Some(64)


Model:
    Name: gemini-1.5-pro-002
    Version: 002
    Display Name: Gemini 1.5 Pro 002
    Description: Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens, released in September of 2024.
    Input Token Limit: 2000000
    Output Token Limit: 8192
    Supported Generation Methods: ["generateContent", "countTokens", "createCachedContent"]
    Temperature: Some(1.0)
    Max Temperature: Some(2.0)
    Top P: Some(0.95)
    Top K: Some(40)


Model:
    Name: gemini-1.5-pro
    Version: 001
    Display Name: Gemini 1.5 Pro
    Description: Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens, released in May of 2024.
    Input Token Limit: 2000000
    Output Token Limit: 8192
    Supported Generation Methods: ["generateContent", "countTokens"]
    Temperature: Some(1.0)
    Max Temperature: Some(2.0)
    Top P: Some(0.95)
    Top K: Some(40)


Model:
    Name: gemini-1.5

    Version: 2.0
    Display Name: Gemini 2.0 Flash (Image Generation) Experimental
    Description: Gemini 2.0 Flash (Image Generation) Experimental
    Input Token Limit: 1048576
    Output Token Limit: 8192
    Supported Generation Methods: ["generateContent", "countTokens", "bidiGenerateContent"]
    Temperature: Some(1.0)
    Max Temperature: Some(2.0)
    Top P: Some(0.95)
    Top K: Some(40)


Model:
    Name: gemini-2.0-flash-lite-001
    Version: 2.0
    Display Name: Gemini 2.0 Flash-Lite 001
    Description: Stable version of Gemini 2.0 Flash Lite
    Input Token Limit: 1048576
    Output Token Limit: 8192
    Supported Generation Methods: ["generateContent", "countTokens"]
    Temperature: Some(1.0)
    Max Temperature: Some(2.0)
    Top P: Some(0.95)
    Top K: Some(40)


Model:
    Name: gemini-2.0-flash-lite
    Version: 2.0
    Display Name: Gemini 2.0 Flash-Lite
    Description: Gemini 2.0 Flash-Lite
    Input Token Limit: 1048576
    Output Token Limit: 8192
    Suppo

    Temperature: Some(0.2)
    Max Temperature: None
    Top P: Some(1.0)
    Top K: Some(40)


Model:
    Name: imagen-3.0-generate-002
    Version: 002
    Display Name: Imagen 3.0 002 model
    Description: Vertex served Imagen 3.0 002 model
    Input Token Limit: 480
    Output Token Limit: 8192
    Supported Generation Methods: ["predict"]
    Temperature: None
    Max Temperature: None
    Top P: None
    Top K: None


Model:
    Name: gemini-2.0-flash-live-001
    Version: 001
    Display Name: Gemini 2.0 Flash 001
    Description: Gemini 2.0 Flash 001
    Input Token Limit: 131072
    Output Token Limit: 8192
    Supported Generation Methods: ["bidiGenerateContent", "countTokens"]
    Temperature: Some(1.0)
    Max Temperature: Some(2.0)
    Top P: Some(0.95)
    Top K: Some(64)



---